In [ ]:
using Revise

In [ ]:
using JuMP, ConditionalJuMP, Gurobi
using Plots; gr()

In [ ]:
m = Model(solver=GurobiSolver())
@variable m -1 <= x <= 1
@variable m -1 <= y <= 1
@implies m (x <= 0) => (y == 0.5)
@constraint m y >= x
d = -1
@objective m Min ((x - d)^2 + (y - (d - 0.5))^2)
solve(m)
getvalue(x), getvalue(y)

In [ ]:
m = Model(solver=GurobiSolver())
@variable m x >= 0.5
@variable m y
@constraint m y >= x
# JuMP.fix(x, 0.5)
@objective m Min x^2 + y^2 + 0.1 * x * y
# c = @constraint m x == -0.5
solve(m)
display(m)
getvalue.((y, x))

In [ ]:
include("explicit_qp.jl")
eq = ExplicitQP

In [ ]:
solution = eq.explicit_solution(m, [x])

In [ ]:
eq.getsolution(solution, y)

In [ ]:
eq.getparameter(solution, x)

In [ ]:
getvalue.((y, x))

In [ ]:
eq.jacobian(eq.getsolution.(solution, [y]))

In [ ]:
m = Model(solver=GurobiSolver())
@variable(m, x[1:2])
JuMP.fix.(x, [0.1, 0])
U = []
X = [x]
N = 20
Δt = 0.1
for i in 1:N
    u = @variable(m, lowerbound=-1, upperbound=1)
    xi = @variable(m, [1:2])
    @constraint(m, xi[2] == X[end][2] + Δt * u)
    @constraint(m, xi[1] == X[end][1] + Δt * xi[2])
    push!(U, u)
    push!(X, xi)
end

@objective m Min sum([x[1]^2 + 0.01 * x[2]^2 for x in X]) + 0.01 * sum([u^2 for u in U])
solve(m)



In [ ]:
plt = plot([x[1] for x in getvalue.(X)], label="x")
plot!(plt, getvalue.(U), label="u")

In [ ]:
ex = eq.explicit_solution(m, X[1])

In [ ]:
eq.getsolution(ex, U[1])